In [ ]:
#dataset_name = "preprocessed_cadec"
dataset_name = "ADE_corpus_output3"

#pretrained_bert = 'SpanBERT/spanbert-base-cased'
#pretrained_bert = 'allenai/scibert_scivocab_uncased'
#pretrained_bert = 'dmis-lab/biobert-base-cased-v1.2'
# retrained_bert = 'cimm-kzn/endr-bert'
#pretrained_bert = 'emilyalsentzer/Bio_ClinicalBERT'
#pretrained_bert = 'dmis-lab/biobert-v1.1'
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract'
#pretrained_bert = 'bert-large-uncased'
pretrained_bert = 'michiyasunaga/BioLinkBERT-large'

In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import numpy as np
import pandas as pd
import datetime
import time
import random
import os
import keras
import wandb
import statistics
from keras.utils import pad_sequences
from transformers import BertTokenizer, BertForSequenceClassification, BertForTokenClassification, AdamW, BertConfig, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification, AutoConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from seqeval.metrics import f1_score,accuracy_score, precision_score, recall_score, classification_report

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv(f"{dataset_name}.csv")

In [ ]:
df.head()

,sentences,labels
0,"['Intravenous', 'azithromycin', '-', 'induced'...","['O', 'B-drug', 'O', 'O', 'B-effect', 'O']"
1,"['Immobilization', ',', 'while', ""Paget's"", 'b...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['Unaccountable', 'severe', 'hypercalcemia', '...","['O', 'O', 'B-effect', 'O', 'O', 'O', 'O', 'O'..."
3,"['METHODS', ':', 'We', 'report', 'two', 'cases...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect'..."
4,"['Naproxen', ',', 'the', 'most', 'common', 'of...","['B-drug', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [ ]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: jon-l-collins. Use `wandb login --relogin` to force relogin


True

In [ ]:
def ret_list(text):
    return eval(text)
df['sentences'] = df['sentences'].apply(ret_list)
df['labels'] = df['labels'].apply(ret_list)

In [ ]:
train, test = train_test_split(df, test_size=0.15, random_state=42)

In [ ]:
len(train), len(test)

(3823, 675)

In [ ]:
sentences = train['sentences'].to_list()
labels = train['labels'].to_list()

In [ ]:
sweep_config = {
    'method': 'random'
    }

In [ ]:
metric = {
        'name': 'val_loss',
        'goal': 'minimize'
}
sweep_config['metric'] = metric

In [ ]:
print(str(datetime.datetime.now()))

2024-02-28 02:24:50.918878


In [ ]:
parameters_dict = {
    'learning_rate': {
        'values': [5e-5,1e-5,5e-6]
    },
    'batch_size': {
        'values': [4,8,16] 
        },
    
    'dropout': {
          'values': [0.1,0.15,0.2,0.3]
        },
    
    'epochs': {
        'values': [17]},#20
    
    'weight_decay': {
        'values': [0.001,0.01]
    }   
 
}
sweep_config['parameters'] = parameters_dict
pretrained_bert2 = pretrained_bert.replace('/','')
sweep_id = wandb.sweep(sweep_config, project=f"NER_{dataset_name}_{pretrained_bert2}_{str(datetime.datetime.now())[:10]}")

Create sweep with ID: zcsnbat3
Sweep URL: https://wandb.ai/jon-l-collins/NER_ADE_corpus_output3_microsoftBiomedNLP-BiomedBERT-large-uncased-abstract_2024-02-28/sweeps/zcsnbat3


In [ ]:
tokenizer = BertTokenizer.from_pretrained(pretrained_bert, do_lower_case=True)

In [ ]:
TokenLength = [len(tokenizer.encode(' '.join(i),add_special_tokens=True)) for i in sentences]

In [ ]:
print("Minimum length: {:,} tokens".format(min(TokenLength)))
print("Maximum length: {:,} tokens".format(max(TokenLength)))
print("Average length: {:,} tokens".format(int(np.median(TokenLength))))

Minimum length: 4 tokens
Maximum length: 103 tokens
Average length: 24 tokens


In [ ]:
unique_labels = set()
for label in labels:
    for l in label:
        unique_labels.add(l)
        
unique_labels 

{'B-drug', 'B-effect', 'I-drug', 'I-effect', 'O'}

In [ ]:
data_tags = list(unique_labels)
data_tags

['B-drug', 'I-drug', 'O', 'I-effect', 'B-effect']

In [ ]:
tag_values = data_tags
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
tag2idx

{'B-drug': 0, 'I-drug': 1, 'O': 2, 'I-effect': 3, 'B-effect': 4, 'PAD': 5}

In [ ]:
print(sentences[0])
print(labels[0])

['OBJECTIVE', ':', 'To', 'report', 'the', 'case', 'of', 'a', 'young', 'woman', 'with', 'Graves', "'", 'disease', 'in', 'whom', 'ototoxicity', 'developed', 'because', 'of', 'propylthiouracil', '(', 'PTU', ')-', 'induced', 'antineutrophil', 'cytoplasmic', 'antibody', '(', 'ANCA)-associated', 'vasculitis', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'O', 'O', 'O', 'O', 'O', 'B-drug', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
def tokenize_and_align_labels(sentences, labels):
    tokenized_sentences = []
    aligned_labels = []
    for word, label in zip(sentences, labels):
        tokenized_word = tokenizer.tokenize(word)
        subwords = len(tokenized_word)
        
        tokenized_sentences.extend(tokenized_word)
        
        if subwords > 1 and label != "O":
            c = label.replace("B-","I-")
            aligned_labels.extend([label] + [c] * (subwords - 1))
        else:
            aligned_labels.extend([label]*subwords)
          
    return tokenized_sentences, aligned_labels

In [ ]:
tokenized_data = [tokenize_and_align_labels(sent, label) for sent, label in zip(sentences, labels)]

In [ ]:
print(tokenized_data[0])

(['objective', ':', 'to', 'report', 'the', 'case', 'of', 'a', 'young', 'woman', 'with', 'graves', "'", 'disease', 'in', 'whom', 'ot', '##otoxicity', 'developed', 'because', 'of', 'propyl', '##thio', '##ura', '##cil', '(', 'pt', '##u', ')', '-', 'induced', 'antine', '##utr', '##ophil', 'cytoplasmic', 'antibody', '(', 'anca', ')', '-', 'associated', 'vasculitis', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'I-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-drug', 'I-drug', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])


In [ ]:
tokenized_texts = [sentandlabel[0] for sentandlabel in tokenized_data]
new_labels = [sentandlabel[1] for sentandlabel in tokenized_data]

In [ ]:
print(tokenized_texts[0])
print(new_labels[0])

['objective', ':', 'to', 'report', 'the', 'case', 'of', 'a', 'young', 'woman', 'with', 'graves', "'", 'disease', 'in', 'whom', 'ot', '##otoxicity', 'developed', 'because', 'of', 'propyl', '##thio', '##ura', '##cil', '(', 'pt', '##u', ')', '-', 'induced', 'antine', '##utr', '##ophil', 'cytoplasmic', 'antibody', '(', 'anca', ')', '-', 'associated', 'vasculitis', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'I-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-drug', 'I-drug', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(text) for text in tokenized_texts],
                          maxlen=130, dtype="long", value=0.0, 
                          truncating="post", padding="post")
label_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in new_labels],maxlen=130, value=tag2idx["PAD"], padding="post",dtype="long", truncating="post")

In [ ]:
print(input_ids[0])
print(label_tags[0])

[ 2833    29  1701  2255  1680  2632  1685    42  3612  6764  1715 19492
    10  2174  1682  6314  7833  9397  2829  3058  1685 14429 23410 10694
 15800    11  4349  1029    12    16  2326 20567  3813  4149  7314  3762
    11 21403    12    16  2138 14479    17     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 3 2 2 2 2 2 2 2 2 0 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5

In [ ]:
print(len(input_ids[0]))
print(len(label_tags[0]))

130
130


In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
train_ids, val_ids, train_tags, val_tags, train_masks, val_masks = train_test_split(input_ids, label_tags, attention_masks, test_size=0.2, random_state=42)

In [ ]:
len(train_ids), len(val_ids)

(3058, 765)

In [ ]:
train_ids

array([[ 1802,  2255,    42, ...,     0,     0,     0],
       [ 5388,  1715, 15748, ...,     0,     0,     0],
       [12771,  1884, 13248, ...,     0,     0,     0],
       ...,
       [15503,  8386,  1705, ...,     0,     0,     0],
       [ 1802,  2143,    42, ...,     0,     0,     0],
       [ 3612,  2564,  5114, ...,     0,     0,     0]])

In [ ]:
train_inputs = torch.tensor(train_ids,dtype=torch.long)

In [ ]:
val_inputs = torch.tensor(val_ids,dtype=torch.long)
train_labels = torch.tensor(train_tags,dtype=torch.long)
val_labels = torch.tensor(val_tags,dtype=torch.long)
train_mask = torch.tensor(train_masks,dtype=torch.long)
val_mask = torch.tensor(val_masks,dtype=torch.long)

In [ ]:
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# batch_size = 12
# train_dataset = TensorDataset(train_inputs, train_mask, train_labels)
# train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)

# validation_dataset = TensorDataset(val_inputs, val_mask, val_labels)
# validation_dataloader = DataLoader(validation_dataset, sampler=SequentialSampler(validation_dataset), batch_size=batch_size)

In [ ]:
def ret_dataloader():
    batch_size = wandb.config.batch_size
    train_dataset = TensorDataset(train_inputs, train_mask, train_labels)
    train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
    
    validation_dataset = TensorDataset(val_inputs, val_mask, val_labels)
    validation_dataloader = DataLoader(validation_dataset, sampler=SequentialSampler(validation_dataset), batch_size=batch_size)
    return train_dataloader, validation_dataloader

In [ ]:
def ret_model():
    
    configuration = AutoConfig.from_pretrained(pretrained_bert)
    configuration.hidden_dropout_prob = wandb.config.dropout
    configuration.attention_probs_dropout_prob = wandb.config.dropout
    configuration.num_labels = len(tag2idx)
    configuration.output_attentions = False
    configuration.output_hidden_states = False

    model = AutoModelForTokenClassification.from_pretrained(pretrained_bert,config=configuration)
    return model

In [ ]:
# from transformers import BertForTokenClassification, AdamW, BertConfig
# model = BertForTokenClassification.from_pretrained(
#     pretrained_bert,
#     num_labels=len(tag2idx),
#     output_attentions = False,
#     output_hidden_states = False
# )

In [ ]:
def ret_optim(model):
    print(f"learning_rate = {wandb.config.learning_rate}")
    optimizer = AdamW(model.parameters(),
                      lr = wandb.config.learning_rate, 
                      eps = 1e-8,weight_decay = wandb.config.weight_decay)
    return optimizer

In [ ]:
def ret_scheduler(optimizer, dataloader_train):
    epochs = wandb.config.epochs
    total_steps = len(dataloader_train) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)
    return scheduler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train():
    wandb.init()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model=ret_model()
    model.to(device)
    dataloader_train, dataloader_validation = ret_dataloader()
    optimizer = ret_optim(model)
    scheduler = ret_scheduler(optimizer, dataloader_train) 
    epochs = wandb.config.epochs 
    loss_values, val_loss = [], []
    for epoch_i in range(epochs):  
        total_loss = 0
        model.train()
        for step, batch in enumerate(dataloader_train):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            model.zero_grad()
            outputs = model(b_input_ids, token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels)
            loss = outputs[0]
            wandb.log({"train_batch_loss":loss.item()})
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        avg_train_loss = total_loss / len(dataloader_train)
        wandb.log({"train_loss": avg_train_loss, "epoch": epoch_i})
        print("Average training loss: {0:.2f}".format(avg_train_loss))
        print("")
        print("Running Validation...")
        model.eval()
        nb_eval_steps, nb_eval_examples = 0, 0
        eval_loss = 0
        predictions , true_labels = [], []
        for batch in dataloader_validation:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            logits = outputs[1].detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
            #print(predictions)
            #predictions.append(logits)
            true_labels.extend(label_ids)
            eval_loss += outputs[0].mean().item()
            nb_eval_examples += b_input_ids.size(0)
            nb_eval_steps += 1
        eval_loss = eval_loss / nb_eval_steps
        wandb.log({"val_loss": eval_loss,"epoch": epoch_i})
        print("Validation loss: {0:.2f}".format(eval_loss))
        
        prediction_tags = []
        validation_tags = []
        for item1,item2  in zip(predictions, true_labels):
          p1,v1= [],[]
          for i in range(len(item2)):
            if tag_values[item2[i]] != "PAD":
              p1.append(tag_values[item1[i]])
              v1.append(tag_values[item2[i]])
          prediction_tags.append(p1)
          validation_tags.append(v1)
        #predictions_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels) for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
        #validation_tags = [tag_values[l_i] for l in true_labels for l_i in l if tag_values[l_i] != "PAD"]
        # predictions = np.concatenate(predictions, axis=0) #added
        # true_vals = np.concatenate(true_labels, axis=0) #added
        # predictions_tags = np.argmax(predictions, axis=2).flatten()
        # validation_tags = true_vals.flatten()
        acc_score = accuracy_score(prediction_tags, validation_tags)
        p_score = precision_score(prediction_tags, validation_tags)
        r_score = recall_score(prediction_tags, validation_tags)
        f1_s = f1_score(prediction_tags, validation_tags)
        
        # f1_score = f1_score(predictions_tags, validation_tags)
        wandb.log({"val_accuracy_score": acc_score,"epoch": epoch_i})
        wandb.log({"precision score": p_score,"epoch": epoch_i})
        wandb.log({"recall score": r_score,"epoch": epoch_i})
        wandb.log({"val_f1_score": f1_s,"epoch": epoch_i})
        
        # wandb.log({"val_f1_score": f1_score,"epoch": epoch_i})
        print("Validation Accuracy: {0:.2f}".format(acc_score))
        print("Validation F1 Score: {0:.2f}".format(f1_s))
        print("Recall Score: {0:.2f}".format(r_score))
        print("Precision Score: {0:.2f}".format(p_score))

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 24oq2wdy with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.1
wandb: 	epochs: 17
wandb: 	learning_rate: 5e-05
wandb: 	weight_decay: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of BertForTokenClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


learning_rate = 5e-05


c:\Users\jonlc\anaconda3\envs\kerasenv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\jonlc\AppData\Local\Temp\ipykernel_22768\3119774215.py:25: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)


Average training loss: 0.10

Running Validation...
Validation loss: 0.03


...

c:\Users\jonlc\anaconda3\envs\kerasenv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\jonlc\AppData\Local\Temp\ipykernel_22768\3119774215.py:25: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)


Average training loss: 0.09

Running Validation...
Validation loss: 0.03
Validation Accuracy: 0.95
Validation F1 Score: 0.84
Recall Score: 0.78
Precision Score: 0.90


c:\Users\jonlc\anaconda3\envs\kerasenv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


Average training loss: 0.02

Running Validation...
Validation loss: 0.02
Validation Accuracy: 0.95
Validation F1 Score: 0.85
Recall Score: 0.82
Precision Score: 0.88
Average training loss: 0.02

Running Validation...
Validation loss: 0.03
Validation Accuracy: 0.95
Validation F1 Score: 0.86
Recall Score: 0.84
Precision Score: 0.88
Average training loss: 0.01

Running Validation...
Validation loss: 0.03
Validation Accuracy: 0.96
Validation F1 Score: 0.87
Recall Score: 0.85
Precision Score: 0.89
Average training loss: 0.01

Running Validation...
Validation loss: 0.03
Validation Accuracy: 0.95
Validation F1 Score: 0.86
Recall Score: 0.83
Precision Score: 0.89
Average training loss: 0.01

Running Validation...
Validation loss: 0.04
Validation Accuracy: 0.96
Validation F1 Score: 0.88
Recall Score: 0.85
Precision Score: 0.90
Average training loss: 0.01

Running Validation...
Validation loss: 0.04
Validation Accuracy: 0.95
Validation F1 Score: 0.87
Recall Score: 0.84
Precision Score: 0.90
Aver

epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇████
precision score,█▁▂▅▆██▆▅▄▄
recall score,▁▅▆▇▆▇▆▇▇▇█
train_batch_loss,▆█▆▅▂▆▃▂▃▂▃▄▁▂▃▃▁▁▄▂▂▄█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▃▂▂▂▁▁▁▁▁▁
val_accuracy_score,▁▆▅█▆█▆▆▆▇▇
val_f1_score,▁▃▅▇▅█▇▇▇▇█
val_loss,▃▁▂▃▄▄▅▆▇▆█
epoch,10
precision score,0.88877
recall score,0.86088


In [ ]:
tokenized_sentence = tokenizer.encode(test_sentences[11])
input_ids_test = torch.tensor([tokenized_sentence]).to(device)
with torch.no_grad():
    output = model(input_ids_test)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
tokens = tokenizer.convert_ids_to_tokens(input_ids_test.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[idx])
        new_tokens.append(token)
new_labels = new_labels[1:-1]
print(new_labels)
print(test_labels[11])

NameError: name 'test_sentences' is not defined

In [ ]:
accuracy_list = []

for i in range(len(test_sentences)):
    tokenized_sentence = tokenizer.encode(test_sentences[i])
    input_ids_test = torch.tensor([tokenized_sentence]).to(device)
    with torch.no_grad():
        output = model(input_ids_test)
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
    tokens = tokenizer.convert_ids_to_tokens(input_ids_test.to('cpu').numpy()[0])
    new_tokens, new_labels = [], []
    for token, idx in zip(tokens, label_indices[0]):
        if token.startswith("##"):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_labels.append(tag_values[idx])
            new_tokens.append(token)
    new_labels = new_labels[1:-1]
    accuracy_list.append(accuracy_score(new_labels,test_labels[i]))
    print("Predicted labels:",new_labels)
    print("Actual labels:",test_labels[i])
    
print("\n----------Done------------")
print("Test Accuracy",statistics.mean(accuracy_list))

Predicted labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'O', 'O', 'O', 'B-drug', 'O']
Actual labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'O', 'O', 'O', 'B-drug', 'O']
Predicted labels: ['B-drug', 'O', 'O', 'B-effect', 'I-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Actual labels: ['B-drug', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'O', 'O', 'O', 'O', 'O', 'O']
Predicted labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Actual labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-drug', 'I-drug', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Predicted labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-effect', 'I-effect', 'O', 'O', 'O', 'O', 'O', 'O', 'B-drug', 'I-drug', 'O', 'B-drug', 'O']
Actual label